### Author : Kubam Ivo
### Purpose: This notebook covers objectives 1 and 5 of the project assignment

In [1]:
# importing necessary modules
import numpy as np 
import pandas as pd
from PIL import Image
import os
import json

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
# initialising hyper paramenters
BATCH = 2 #the number of data samples propagated through the network before the parameters are updated
EPOCHS = 10 #the number times to iterate over the dataset

LR = 0.01
IM_SIZE = 32

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # checking if GPU is available if not use CPU
torch.set_num_threads(1) # number of threads to one
TRAIN_DIR = "C:\\Users\\ivomb\\OneDrive\\Msc Data Science\\Second-Semester\\INFO-H-515-BigData-Distributed-Data-Management-and-Scalable-Analytics\\Practical_Sessions\\mini_herbarium\\train\\"    # training source
Test_Dir = "C:\\Users\\ivomb\\Downloads\\test\\"


C:\Users\ivomb\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# loading train images metadata
with open(TRAIN_DIR + 'metadata.json', "r", encoding="ISO-8859-1") as file:
    train = json.load(file) 

In [4]:

# importing image file name and their annotations as pandas dataframe
train_img = pd.DataFrame(train['images'])
train_ann = pd.DataFrame(train['annotations']).drop(columns='image_id')
train_df = train_img.merge(train_ann, on='id') # merging dataframes

In [547]:
train_df.head()

,file_name,height,id,license,width,category_id,institution_id
0,images/000/01/404873.jpg,1000,404873,0,680,1,0
1,images/000/01/1515603.jpg,1000,1515603,0,680,1,0
2,images/000/00/1433074.jpg,1000,1433074,0,660,0,0
3,images/000/01/1104517.jpg,1000,1104517,0,680,1,0
4,images/000/01/1948744.jpg,1000,1948744,0,680,1,0


In [548]:
NUM_CL = len(train_df['category_id'].value_counts()) # requesting number of classes


In [549]:
X, y = train_df['file_name'].values, train_df['category_id'].values # separating features and labels

In [550]:
# Split into Train and test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [551]:
Transform = transforms.Compose(
    [
    transforms.Resize((IM_SIZE, IM_SIZE)), # Resize the input image to the given size 
    transforms.ToTensor(), # ToTensor converts a PIL image or NumPy ndarray into a FloatTensor and scales the image’s pixel intensity values in the range [0., 1.]
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]) # Normalize a tensor image with mean and standard deviation. mean[1],...,mean[n]) and std: (std[1],..,std[n]) for n channels


In [552]:
# Creating a custom dataset for image files
#The Dataset retrieves our dataset’s features and labels one sample at a time
class GetData(Dataset):
    
    def __init__(self, Dir, FNames, Labels, Transform): # initialising class attributes
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self): # returns number of samples
        return len(self.fnames)

    def __getitem__(self, index):        # retrives image sample
        x = Image.open(os.path.join(self.dir, self.fnames[index])) #opening images with pilow
        # transform
        return self.transform(x), self.labels[index]
        

In [553]:
trainset = GetData(TRAIN_DIR, X_train, y_train, Transform) # training dataset
trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True) # preparing data for taining

testset = GetData(TRAIN_DIR, X_test, y_test, Transform) # test dataset
testloader = DataLoader(testset, batch_size=1, shuffle=True) # preparing data for test

# While training a model, we typically want to pass samples in “minibatches”, reshuffle the data at every epoch to reduce model overfitting, and use Python’s multiprocessing to speed up data retrieval.

### Modelling

In [554]:
#Model 1
model = torchvision.models.vgg11(num_classes=NUM_CL) # instantiating the resnet34 neural network model
model = model.to(DEVICE)

In [555]:
#model 2
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, NUM_CL)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()

In [556]:
criterion = nn.CrossEntropyLoss() # loss function

# Choose 1 optimiser: hold the current model state and will update the parameters based on the computed gradients.
optimizer = torch.optim.Adam(model.parameters(), lr=LR) # Adam optimisation
#optimizer = torch.optim.Adagrad(model.parameters(), lr=LR)

In [557]:
%%time
# Training Mode
for epoch in range(EPOCHS):
    tr_loss = 0.0
    correct = 0
    model = model.train()

    for i, (images, labels) in enumerate(trainloader):        
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)       
        logits = model(images)       
        loss = criterion(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        tr_loss += loss.detach().item()        

    
    model.eval()
    print('Epoch: %d | Loss: %.4f'%(epoch, tr_loss ))

Epoch: 0 | Loss: 3.5604
Epoch: 1 | Loss: 1.8620
Epoch: 2 | Loss: 1.7588
Epoch: 3 | Loss: 1.6412
Epoch: 4 | Loss: 1.1656
Epoch: 5 | Loss: 1.4680
Epoch: 6 | Loss: 0.8917
Epoch: 7 | Loss: 0.6738
Epoch: 8 | Loss: 0.7386
Epoch: 9 | Loss: 0.5186
Wall time: 1.08 s


In [558]:
# Testing mode
for epoch in range(EPOCHS):
    tr_loss = 0.0
    correct = 0
    model = model.eval()

    for i, (images, labels) in enumerate(testloader):        
        images = images.to(DEVICE)
        
        labels = labels.to(DEVICE)         

        # Get the predictions
        out = model(images)
        # Calculate the accuracy
        _, predicted = torch.max(out.data, 1)
        

        if labels == predicted:
            correct += torch.sum(labels==predicted).item()

    
    model.eval()
    print('Epoch: %d | Accuracy: %.4f'%(epoch, correct/len(testloader.dataset) ))

Epoch: 0 | Accuracy: 0.5000
Epoch: 1 | Accuracy: 0.5000
Epoch: 2 | Accuracy: 0.5000
Epoch: 3 | Accuracy: 0.5000
Epoch: 4 | Accuracy: 0.5000
Epoch: 5 | Accuracy: 0.5000
Epoch: 6 | Accuracy: 0.5000
Epoch: 7 | Accuracy: 0.5000
Epoch: 8 | Accuracy: 0.5000
Epoch: 9 | Accuracy: 0.5000


In [559]:
torch.save(model, 'model.pth')

## Prediction

In [560]:
# Loading the saved model
model_1 = torch.load('model.pth')

#Sample image for prediction
x = Transform(Image.open("C:\\Users\\ivomb\\OneDrive\\Msc Data Science\\Second-Semester\\INFO-H-515-BigData-Distributed-Data-Management-and-Scalable-Analytics\\Practical_Sessions\\mini_herbarium\\train\\images\\000\\01\\230965.jpg"))

x= x.unsqueeze(0) 
x.shape

torch.Size([1, 3, 32, 32])

In [561]:
#current class
train_df["category_id"][train_df['file_name'] == "images/000/01/230965.jpg"].values

array([1], dtype=int64)

In [562]:
# prediction
output = model_1(x)

In [563]:
# Requesting the class with the highest prediction probability
_, predicted = torch.max(output, 1)

In [564]:
#predicted class
predicted

tensor([1])

In [565]:
# Image Two
x2 =  Transform(Image.open("C:\\Users\\ivomb\\OneDrive\\Msc Data Science\\Second-Semester\\INFO-H-515-BigData-Distributed-Data-Management-and-Scalable-Analytics\\Practical_Sessions\\mini_herbarium\\train\\images\\000\\00\\1360648.jpg"))
x2= x2.unsqueeze(0) 

In [566]:
#current class
train_df["category_id"][train_df['file_name'] == "images/000/00/1360648.jpg"]

5    0
Name: category_id, dtype: int64

In [567]:
# prediction
output = model_1(x2)

In [568]:
# Requesting the class with the highest prediction probability
_, predicted = torch.max(output, 1)

In [569]:
#predicted class
predicted

tensor([1])